In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
xlsx = pd.ExcelFile('TJ_list.xlsx')
tj_players = pd.read_excel(xlsx, 'TJ List')

In [3]:
tj_players.head()

,Player,TJ Surgery Date,Team,Level,Position,Throws,Country,High School,College(s),Age,...,IP,K-BB%,ERA-,FIP-,G.1,GS.1,IP.1,K-BB%.1,ERA-.1,FIP-.1
0,Colin Rea,2016-11-10,SD,MLB,P,R,United States,Iowa,Indiana State,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bryan Harper,2016-11-08,WAS,AAA,P,L,United States,Nevada,"Southern Nevada, South Carolina",26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yimi Garcia,2016-10-25,LAD,AA,P,R,Dominican,NaN,NaN,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ryan Cook,2016-10-20,SEA,Rk,P,R,United States,California,USC,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,J.B. Wendelken,2016-10-12,OAK,MLB,P,R,United States,Georgia,Middle Georgia,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
tj_players.columns

Index([u'Player', u'TJ Surgery Date', u'Team', u'Level', u'Position',
       u'Throws', u'Country', u'High School', u'College(s)', u'Age',
       u'Return Date (same level)', u'Recovery Time (months)', u'mlbamid',
       u'fgid', u'Surgeon(s)', u'Post-TJ MLB G', u'Post-TJ MLB IP/PA',
       u'Active', u'Year', u'Month', u'Started\nThrowing', u'Mound',
       u'Bullpen', u'Live\nHitters', u'Game', u'Setback', u'Setback Type',
       u'Setback.1', u'Setback Type.1', u'G', u'GS', u'IP', u'K-BB%', u'ERA-',
       u'FIP-', u'G.1', u'GS.1', u'IP.1', u'K-BB%.1', u'ERA-.1', u'FIP-.1'],
      dtype='object')

In [5]:
tj_players.isnull().sum()

Player                         0
TJ Surgery Date                0
Team                           0
Level                          0
Position                       0
Throws                         0
Country                        0
High School                  289
College(s)                   569
Age                            0
Return Date (same level)     295
Recovery Time (months)       723
mlbamid                       14
fgid                          42
Surgeon(s)                   876
Post-TJ MLB G                  0
Post-TJ MLB IP/PA              0
Active                         0
Year                           0
Month                          0
Started\nThrowing           1321
Mound                       1329
Bullpen                     1325
Live\nHitters               1324
Game                        1311
Setback                     1317
Setback Type                1317
Setback.1                   1330
Setback Type.1              1330
G                           1276
GS        

In [6]:
del tj_players['High School']
del tj_players['College(s)']
del tj_players['Return Date (same level)']
del tj_players['Recovery Time (months)']
del tj_players['mlbamid']
del tj_players['Surgeon(s)']
del tj_players['Started\nThrowing']
del tj_players['Mound']
del tj_players['Bullpen']
del tj_players['Live\nHitters']
del tj_players['Game']
del tj_players['Setback']
del tj_players['Setback Type']
del tj_players['Setback.1']
del tj_players['Setback Type.1']
del tj_players['G']
del tj_players['GS']
del tj_players['IP']
del tj_players['K-BB%']
del tj_players['ERA-']
del tj_players['FIP-']
del tj_players['G.1']
del tj_players['GS.1']
del tj_players['IP.1']
del tj_players['K-BB%.1']
del tj_players['ERA-.1']
del tj_players['FIP-.1']


In [11]:
tj_players.head()

,player,tj_surgery_date,team,level,position,throws,country,age,fgid,post-tj_mlb_g,post-tj_mlb_ip/pa,active,year,month,tjs
0,Colin Rea,2016-11-10,SD,MLB,P,R,United States,25,12317,0,0.0,1,2016,11,1
1,Bryan Harper,2016-11-08,WAS,AAA,P,L,United States,26,sa549768,0,0.0,1,2016,11,1
2,Yimi Garcia,2016-10-25,LAD,AA,P,R,Dominican,25,12095,0,0.0,1,2016,10,1
3,Ryan Cook,2016-10-20,SEA,Rk,P,R,United States,29,8855,0,0.0,1,2016,10,1
4,J.B. Wendelken,2016-10-12,OAK,MLB,P,R,United States,23,14506,0,0.0,1,2016,10,1


In [8]:
tj_players.isnull().sum()

Player                0
TJ Surgery Date       0
Team                  0
Level                 0
Position              0
Throws                0
Country               0
Age                   0
fgid                 42
Post-TJ MLB G         0
Post-TJ MLB IP/PA     0
Active                0
Year                  0
Month                 0
dtype: int64

In [9]:
tj_players['tjs'] = 1

In [10]:
tj_players.columns = [i.replace(' ','_').lower() for i in tj_players.columns]

In [16]:
tj_players['player']=tj_players['player'].apply(lambda x: x.replace(' ','_').lower())

In [17]:
tj_players.head()

,player,tj_surgery_date,team,level,position,throws,country,age,fgid,post-tj_mlb_g,post-tj_mlb_ip/pa,active,year,month,tjs
0,colin_rea,2016-11-10,SD,MLB,P,R,United States,25,12317,0,0.0,1,2016,11,1
1,bryan_harper,2016-11-08,WAS,AAA,P,L,United States,26,sa549768,0,0.0,1,2016,11,1
2,yimi_garcia,2016-10-25,LAD,AA,P,R,Dominican,25,12095,0,0.0,1,2016,10,1
3,ryan_cook,2016-10-20,SEA,Rk,P,R,United States,29,8855,0,0.0,1,2016,10,1
4,j.b._wendelken,2016-10-12,OAK,MLB,P,R,United States,23,14506,0,0.0,1,2016,10,1


In [18]:
tj_players['player']=tj_players['player'].apply(lambda x: x.replace('.',''))

In [19]:
tj_players.head()

,player,tj_surgery_date,team,level,position,throws,country,age,fgid,post-tj_mlb_g,post-tj_mlb_ip/pa,active,year,month,tjs
0,colin_rea,2016-11-10,SD,MLB,P,R,United States,25,12317,0,0.0,1,2016,11,1
1,bryan_harper,2016-11-08,WAS,AAA,P,L,United States,26,sa549768,0,0.0,1,2016,11,1
2,yimi_garcia,2016-10-25,LAD,AA,P,R,Dominican,25,12095,0,0.0,1,2016,10,1
3,ryan_cook,2016-10-20,SEA,Rk,P,R,United States,29,8855,0,0.0,1,2016,10,1
4,jb_wendelken,2016-10-12,OAK,MLB,P,R,United States,23,14506,0,0.0,1,2016,10,1


In [21]:
tj_players.to_csv('tj_list.csv', encoding ='utf-8')

In [168]:
fg_info1 = pd.read_csv('../Data/pitchers_standard_2014.csv')

In [169]:
fg_info2 = pd.read_csv('../Data/pitchers_advanced_2014.csv')

In [170]:
fg_info3 = pd.read_csv('../Data/pitchers_battedball_2014.csv')

In [171]:
fg_info4 = pd.read_csv('../Data/pitchers_pitchfxhmov_2014.csv')

In [172]:
fg_info5 = pd.read_csv('../Data/pitchers_pitchfxvmov_2014.csv')

In [173]:
fg_info6 = pd.read_csv('../Data/pitchers_pitchfxplatedisci_2014.csv')

In [174]:
fg_info7 = pd.read_csv('../Data/pitchers_pitchfxvelo_2014.csv')

In [175]:
fg_info8 = pd.read_csv('../Data/pitchers_pitchfxtype_2014.csv')

In [176]:
fg_info9 = pd.read_csv('../Data/pitchers_pitchtype_2014.csv')

In [177]:
fg_info10 = pd.read_csv('../Data/pitchers_pitchvalue_2014.csv')

In [178]:
fg_info11 = pd.read_csv('../Data/pitchers_platedisci_2014.csv')

In [179]:
fg_info2.shape

(545, 22)

In [180]:
merge1 = fg_info1.merge(fg_info2,on='playerid')
merge2 = merge1.merge(fg_info3,on='playerid')
merge3 = merge2.merge(fg_info4,on='playerid')
merge4 = merge3.merge(fg_info5,on='playerid')
merge5 = merge4.merge(fg_info6,on='playerid')
merge6 = merge5.merge(fg_info7,on='playerid')
merge7 = merge6.merge(fg_info8,on='playerid')
merge8 = merge7.merge(fg_info9,on='playerid')
merge9 = merge8.merge(fg_info10,on='playerid')
pitchers_2014 = merge9.merge(fg_info11,on='playerid')

In [181]:
pitchers_2014.shape

(545, 186)

In [182]:
pitchers_2014.to_csv('pitchers_2014.csv')

In [67]:
tj_dict = {}
for index, row in tj_players.iterrows():
    tj_dict[str(row['fgid'])] = {'year':int(row['year']),
                           'month':int(row['month']),
                           'age': int(row['age'])
                           }

In [68]:
tj_dict

{'1142': {'age': 23, 'month': 7, 'year': 2003},
 'sa828659': {'age': 18, 'month': 3, 'year': 2015},
 '1718': {'age': 23, 'month': 11, 'year': 2004},
 'sa505589': {'age': 24, 'month': 1, 'year': 2013},
 '9073': {'age': 19, 'month': 1, 'year': 2006},
 'sa553380': {'age': 23, 'month': 5, 'year': 2012},
 '452': {'age': 31, 'month': 1, 'year': 2006},
 'sa701852': {'age': 23, 'month': 1, 'year': 2015},
 '14914': {'age': 19, 'month': 8, 'year': 2011},
 '11543': {'age': 20, 'month': 1, 'year': 2009},
 'sa549763': {'age': 24, 'month': 3, 'year': 2016},
 '57': {'age': 30, 'month': 7, 'year': 2006},
 '13300': {'age': 22, 'month': 1, 'year': 2013},
 'sa597117': {'age': 20, 'month': 5, 'year': 2014},
 '1060': {'age': 24, 'month': 8, 'year': 2001},
 'sa549768': {'age': 26, 'month': 11, 'year': 2016},
 '14077': {'age': 24, 'month': 10, 'year': 2013},
 '4782': {'age': 21, 'month': 7, 'year': 2006},
 'sa875195': {'age': 20, 'month': 8, 'year': 2016},
 '12520': {'age': 26, 'month': 10, 'year': 2016},
 '

In [195]:
pitchers_2015.head()

,"﻿""Name""_x",Team_x,W,L,ERA_x,G,GS,CG,ShO,SV,HLD,BS,IP_x,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,playerid,"﻿""Name""_y",Team_y,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP_x,LOB%,ERA-,FIP-,xFIP-,ERA_y,FIP,E-F,xFIP,SIERA,"﻿""Name""_x",Team_x,BABIP_y,GB/FB,LD%,GB%,FB%_x,IFFB%,HR/FB,RS,RS/9,Balls,Strikes,Pitches,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,"﻿""Name""_y",Team_y,IP_y,FA-X,FT-X,FC-X,FS-X,FO-X,SI-X,SL-X,CU-X,KC-X,EP-X,CH-X,SC-X,KN-X,"﻿""Name""_x",Team_x,IP_x,FA-Z,FT-Z,FC-Z,FS-Z,FO-Z,SI-Z,SL-Z,CU-Z,KC-Z,EP-Z,CH-Z,SC-Z,KN-Z,"﻿""Name""_y",Team_y,IP_y,O-Swing%_x,Z-Swing%_x,Swing%_x,O-Contact%_x,Z-Contact%_x,Contact%_x,Zone%_x,Pace,"﻿""Name""_x",Team_x,IP_x,vFA,vFT,vFC,vFS,vFO,vSI,vSL,vCU,vKC,vEP,vCH,vSC,vKN,"﻿""Name""_y",Team_y,IP_y,FA%,FT%,FC%,FS%,FO%,SI%,SL%_x,CU%,KC%,EP%,CH%_x,SC%,KN%_x,UN%,"﻿""Name""_x",Team_x,FB%_y,FBv,SL%_y,SLv,CT%,CTv,CB%,CBv,CH%_y,CHv,SF%,SFv,KN%_y,KNv,XX%,"﻿""Name""_y",Team_y,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,"﻿""Name""",Team,O-Swing%_y,Z-Swing%_y,Swing%_y,O-Contact%_y,Z-Contact%_y,Contact%_y,Zone%_y,F-Strike%,SwStr%
0,Adam Ottavino,Rockies,1,0,0.00,10,0,0,0,3,3,0,10.1,35,3,0,0,0,2,0,1,0,0,13,1247,Adam Ottavino,Rockies,11.32,1.74,6.50,0.00,37.1 %,5.7 %,31.4 %,0.094,0.48,0.158,100.0 %,0,36,61,0.00,1.49,-1.49,2.35,1.62,Adam Ottavino,Rockies,0.158,2.00,5.3 %,63.2 %,31.6 %,33.3 %,0.0 %,1,0.87,52,90,142,36.8 %,31.6 %,31.6 %,26.3 %,57.9 %,15.8 %,Adam Ottavino,Rockies,10.1,-5.4,-9.1,1.2,NaN,NaN,NaN,5.9,NaN,NaN,NaN,NaN,NaN,NaN,Adam Ottavino,Rockies,10.1,6.1,3.6,0.5,NaN,NaN,NaN,-1.4,NaN,NaN,NaN,NaN,NaN,NaN,Adam Ottavino,Rockies,10.1,36.4 %,47.7 %,41.6 %,53.6 %,83.9 %,69.5 %,45.8 %,23.8,Adam Ottavino,Rockies,10.1,94.9,95.2,88.8,NaN,NaN,NaN,83.8,NaN,NaN,NaN,NaN,NaN,NaN,Adam Ottavino,Rockies,10.1,38.0 %,17.6 %,3.5 %,NaN,NaN,NaN,40.9 %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adam Ottavino,Rockies,48.6 %,95.7,39.4 %,83.7,12.0 %,89.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adam Ottavino,Rockies,3.3,2.2,-0.4,NaN,NaN,NaN,NaN,4.85,3.90,-2.30,NaN,NaN,NaN,NaN,Adam Ottavino,Rockies,35.1 %,49.2 %,41.6 %,51.9 %,84.4 %,69.5 %,45.8 %,51.4 %,12.7 %
1,Dalier Hinojosa,- - -,2,0,0.73,19,0,0,0,0,3,1,24.2,102,15,3,2,1,11,1,1,1,0,23,15671,Dalier Hinojosa,- - -,8.39,4.01,2.09,0.36,22.6 %,10.8 %,11.8 %,0.167,1.05,0.212,93.8 %,19,82,111,0.73,3.26,-2.53,4.34,3.86,Dalier Hinojosa,- - -,0.212,1.15,13.4 %,46.3 %,40.3 %,7.4 %,3.7 %,17,6.20,159,270,429,34.3 %,28.4 %,37.3 %,10.5 %,67.2 %,22.4 %,Dalier Hinojosa,- - -,24.2,-1.2,-4.9,0.4,NaN,NaN,NaN,2.0,NaN,NaN,NaN,-5.6,NaN,NaN,Dalier Hinojosa,- - -,24.2,11.4,10.7,8.5,NaN,NaN,NaN,0.8,NaN,NaN,NaN,6.5,NaN,NaN,Dalier Hinojosa,- - -,24.2,28.7 %,64.8 %,47.1 %,66.7 %,84.3 %,79.0 %,50.8 %,22.5,Dalier Hinojosa,- - -,24.2,92.4,93.2,89.7,NaN,NaN,NaN,85.4,NaN,NaN,NaN,86.2,NaN,NaN,Dalier Hinojosa,- - -,24.2,20.2 %,43.8 %,1.2 %,NaN,NaN,NaN,22.8 %,NaN,NaN,NaN,12.0 %,NaN,NaN,NaN,Dalier Hinojosa,- - -,63.8 %,93.1,23.5 %,85.3,NaN,NaN,NaN,NaN,12.7 %,86.6,NaN,NaN,NaN,NaN,0.9 %,Dalier Hinojosa,- - -,4.0,0.8,NaN,NaN,0.4,NaN,NaN,1.47,0.75,NaN,NaN,0.69,NaN,NaN,Dalier Hinojosa,- - -,24.8 %,72.4 %,47.1 %,67.9 %,84.0 %,79.5 %,46.8 %,54.9 %,9.6 %
2,Jordan Walden,Cardinals,0,1,0.87,12,0,0,0,1,8,0,10.1,42,7,1,1,0,4,1,0,2,0,12,3271,Jordan Walden,Cardinals,10.45,3.48,3.00,0.00,28.6 %,9.5 %,19.1 %,0.184,1.06,0.269,90.9 %,23,52,80,0.87,1.97,-1.10,3.12,3.00,Jordan Walden,Cardinals,0.269,1.38,26.9 %,42.3 %,30.8 %,25.0 %,0.0 %,1,0.87,50,106,156,38.5 %,23.1 %,38.5 %,11.5 %,65.4 %,23.1 %,Jordan Walden,Cardinals,10.1,-8.6,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,-10.7,NaN,NaN,Jordan Walden,Cardinals,10.1,9.5,NaN,NaN,NaN,NaN,NaN,1.9,NaN,NaN,NaN,5.0,NaN,NaN,Jordan Walden,Cardinals,10.1,46.2 %,66.7 %,54.6 %,45.2 %,78.6 %,61.9 %,40.9 %,23.1,Jordan Walden,Cardinals,10.1,94.4,NaN,NaN,NaN,NaN,NaN,83.9,NaN,NaN,NaN,85.9,NaN,NaN,Jordan Walden,Cardinals,10.1,52.0 %,NaN,NaN,NaN,NaN,NaN,27.3 %,NaN,NaN,NaN,20.8 %,NaN,NaN,NaN,Jordan Walden,Cardinals,54.6 %,94.2,27.3 %,84.0,NaN,NaN,NaN,NaN,18.2 %,85.1,NaN,NaN,NaN,NaN,1.3 %,Jordan Walden,Car

In [ ]:
pitchers_2015['playerid'] = pitchers_2015['playerid'].apply(lambda x: str(x))

In [ ]:
for index, row in pitchers_2015.iterrows():
    if str(row['playerid']) in tj_dict and tj_dict[row['playerid']]['year'] == 2016:
        pitchers_2015.loc[index, 'tj'] = 1
    else:
        pitchers_2015.loc[index, 'tj'] = 0

In [ ]:
for index, row in pitchers_2015.iterrows():
    if row['playerid'] in tj_dict:
        pitchers_2015.loc[index, 'prev_tj'] = 1
    else:
        pitchers_2015.loc[index, 'prev_tj'] = 0

In [ ]:
tj_players[tj_players['level'] == 'MLB'].year.value_counts()

In [ ]:
pitchers_2015[pitchers_2015['prev_tj'] == 1]

In [ ]:
pitchers_2015.corr()

In [ ]:
for n in range(0,len(pitchers_2015.corr()['tj'])):
    if pitchers_2015.corr()['tj'][n] > 0.05:
        print pitchers_2015.corr()['tj'][n:n+1]

In [ ]:
sns.heatmap(pitchers_2015[['FS-X','KC-Z','vKC','tj','prev_tj']].corr())

In [253]:
pitchers_2014.isnull().sum()

W                 0
L                 0
G                 0
GS                0
CG                0
ShO               0
SV                0
HLD               0
BS                0
TBF               0
H                 0
R                 0
ER                0
HR                0
BB                0
IBB               0
HBP               0
WP                0
BK                0
SO                0
playerid          0
K/9               0
BB/9              0
K/BB              0
HR/9              0
K%                0
BB%               0
K-BB%             0
AVG               0
WHIP              0
LOB%              0
ERA-              0
FIP-              0
xFIP-             0
FIP               0
E-F               0
xFIP              0
SIERA             0
GB/FB             0
LD%               0
GB%               0
FB%_x             0
IFFB%             0
HR/FB             0
RS                0
RS/9              0
Balls             0
Strikes           0
Pitches           0
Pull%             0


In [236]:

for index, row in pitchers_2014.iterrows():
    pitchers_2014.loc[index,'IP'] = np.mean(pitchers_2014.loc[index,'IP_x'])

In [237]:
pitchers_2014['BABIP'] = pitchers_2014['BABIP_x'] 

In [238]:
pitchers_2014['ERA'] = pitchers_2014['ERA_x'] 

In [252]:
del pitchers_2014['ERA_y']

In [254]:
pitchers_2014.to_csv('pitchers_2014.csv')

In [255]:
pitchers_2015.shape

(598, 159)

In [256]:
pitchers_2014.shape

(545, 159)

In [113]:
pitchers_2014 = pd.read_csv('pitchers_2014.csv')

In [139]:
pitchers_2014_15_avg = pd.read_csv('pitchers_2015.csv')

In [8]:
pitchers_2014.columns

Index([u'Unnamed: 0', u'W', u'L', u'G', u'GS', u'CG', u'ShO', u'SV', u'HLD',
       u'BS',
       ...
       u'Swing%_y', u'O-Contact%_y', u'Z-Contact%_y', u'Contact%_y',
       u'Zone%_y', u'F-Strike%', u'SwStr%', u'IP', u'BABIP', u'ERA'],
      dtype='object', length=160)

In [49]:
pitchers_2014_15_avg.head()

,Unnamed: 0,W,L,G,GS,CG,ShO,SV,HLD,BS,...,Swing%_y,O-Contact%_y,Z-Contact%_y,Contact%_y,Zone%_y,F-Strike%,SwStr%,IP,BABIP,ERA
0,0,1,0,10,0,0,0,3,3,0,...,41.6 %,51.9 %,84.4 %,69.5 %,45.8 %,51.4 %,12.7 %,10.1,0.158,0.00
1,1,2,0,19,0,0,0,0,3,1,...,47.1 %,67.9 %,84.0 %,79.5 %,46.8 %,54.9 %,9.6 %,24.2,0.212,0.73
2,2,0,1,12,0,0,0,1,8,0,...,54.6 %,47.4 %,73.9 %,61.9 %,42.2 %,69.1 %,20.5 %,10.1,0.269,0.87
3,3,1,0,32,0,0,0,0,3,0,...,47.1 %,64.6 %,87.0 %,77.2 %,42.0 %,61.8 %,10.7 %,39.1,0.192,0.92
4,4,8,1,69,0,0,0,17,18,1,...,46.2 %,60.4 %,82.5 %,74.7 %,45.4 %,61.0 %,11.6 %,67.1,0.200,0.94


In [27]:
np.mean([pitchers_2014_15_avg.iloc[0,1],pitchers_2014.iloc[0,1]])

0.5

In [58]:
type(pitchers_2014_15_avg.iloc[0,155])

str

In [118]:
pitchers_2014['HR/FB']

0       0.0
1       4.5
2      10.0
3       0.0
4       0.0
5       6.3
6       3.8
7       2.8
8       8.3
9       7.7
10      5.9
11      6.9
12      0.0
13      3.8
14      6.7
15      5.7
16      0.0
17      4.8
18      6.7
19      4.9
20      8.1
21     17.4
22      4.3
23      6.8
24      9.5
25      0.0
26      6.6
27     15.4
28      9.5
29      8.0
       ... 
515    12.2
516    35.3
517     2.6
518    20.0
519    27.3
520    13.5
521     6.7
522    25.0
523    27.3
524    17.6
525     8.3
526    19.3
527     9.3
528     8.3
529    12.5
530     0.0
531    11.9
532    15.2
533    21.4
534    11.6
535    26.1
536    25.0
537    22.2
538    24.0
539    13.8
540    18.5
541    13.3
542    18.2
543    20.7
544    23.1
Name: HR/FB, dtype: float64

In [3]:
def f(x):
    if x > 0 and type(x) is not float:
        return float(x.split(' %')[0])

In [115]:
cols2014 = pitchers_2014.columns.tolist()

In [116]:
for c in cols2014:
    if '%' in c:
        pitchers_2014[c] = pitchers_2014[c].apply(f)


In [117]:
pitchers_2014['HR/FB'] = pitchers_2014['HR/FB'].apply(f)
pitchers_2014['playerid'] = pitchers_2014['playerid'].apply(lambda x:str(x))

In [125]:
del cols2014[20]

In [140]:
cols2014_15 = pitchers_2014_15_avg.columns.tolist()
for c in cols2014_15:
    if '%' in c:
        pitchers_2014_15_avg[c] = pitchers_2014_15_avg[c].apply(f)

In [99]:
pitchers_2014.loc[pitchers_2014['playerid'] == row['playerid'],cols2014[7]][0]

2

In [141]:
pitchers_2014_15_avg['HR/FB'] = pitchers_2014_15_avg['HR/FB'].apply(f)
pitchers_2014_15_avg['playerid'] = pitchers_2014_15_avg['playerid'].apply(lambda x:str(x))

In [130]:
p_2014_dict = {}
for index, row in pitchers_2014.iterrows():
    p_2014_dict[row['playerid']] = {cols2014[0]: float(row[cols2014[0]])}
    for n in range(1,len(cols2014)):
        if type(pitchers_2014[cols2014[n]][0]) != str:
            p_2014_dict[row['playerid']].update({cols2014[n] : float(pitchers_2014.loc[pitchers_2014['playerid'] == row['playerid'],cols2014[n]])})


In [127]:
pitchers_2014[cols2014[42]]

0       0.0
1       4.5
2      10.0
3       0.0
4       0.0
5       6.3
6       3.8
7       2.8
8       8.3
9       7.7
10      5.9
11      6.9
12      0.0
13      3.8
14      6.7
15      5.7
16      0.0
17      4.8
18      6.7
19      4.9
20      8.1
21     17.4
22      4.3
23      6.8
24      9.5
25      0.0
26      6.6
27     15.4
28      9.5
29      8.0
       ... 
515    12.2
516    35.3
517     2.6
518    20.0
519    27.3
520    13.5
521     6.7
522    25.0
523    27.3
524    17.6
525     8.3
526    19.3
527     9.3
528     8.3
529    12.5
530     0.0
531    11.9
532    15.2
533    21.4
534    11.6
535    26.1
536    25.0
537    22.2
538    24.0
539    13.8
540    18.5
541    13.3
542    18.2
543    20.7
544    23.1
Name: HR/FB, dtype: float64

In [7]:
pitchers_2014.head()

,Unnamed: 0,W,L,G,GS,CG,ShO,SV,HLD,BS,...,Swing%_y,O-Contact%_y,Z-Contact%_y,Contact%_y,Zone%_y,F-Strike%,SwStr%,IP,BABIP,ERA
0,0,0,0,15,0,0,0,0,2,0,...,48.5,70.5,83.6,77.8,37.8,66.0,10.8,12.1,0.313,0.73
1,1,0,0,23,0,0,0,0,0,0,...,46.5,71.4,82.9,78.7,42.5,61.1,9.9,21.0,0.344,0.86
2,2,0,1,3,1,0,0,0,0,0,...,56.0,77.8,90.4,86.1,49.7,66.7,7.8,10.1,0.188,0.87
3,3,2,1,30,0,0,0,0,9,1,...,42.3,61.5,92.1,82.7,47.7,56.6,7.2,19.0,0.264,0.95
4,4,9,2,71,0,0,0,3,33,3,...,46.6,52.1,77.9,69.3,47.0,60.9,14.3,72.0,0.264,1.00


In [134]:
p_2014_dict['10855']

{'AVG': 0.213,
 'BABIP': 0.313,
 'BB': 3.0,
 'BB%': 6.0,
 'BB/9': 2.19,
 'BK': 0.0,
 'BS': 0.0,
 'Balls': 68.0,
 'CB%': nan,
 'CBv': nan,
 'CG': 0.0,
 'CH%_x': 8.3,
 'CH%_y': nan,
 'CH-X': 5.4,
 'CH-Z': 0.8,
 'CHv': nan,
 'CT%': nan,
 'CTv': nan,
 'CU%': 2.9,
 'CU-X': 2.4,
 'CU-Z': -0.6,
 'Cent%': 34.4,
 'Contact%_x': 77.8,
 'Contact%_y': 77.8,
 'E-F': -0.7,
 'EP%': nan,
 'EP-X': nan,
 'EP-Z': nan,
 'ER': 1.0,
 'ERA': 0.73,
 'ERA-': 20.0,
 'F-Strike%': 66.0,
 'FA%': 55.9,
 'FA-X': 6.7,
 'FA-Z': 10.4,
 'FB%_x': 35.5,
 'FB%_y': 62.8,
 'FBv': 91.5,
 'FC%': nan,
 'FC-X': nan,
 'FC-Z': nan,
 'FIP': 1.43,
 'FIP-': 39.0,
 'FO%': nan,
 'FO-X': nan,
 'FO-Z': nan,
 'FS%': nan,
 'FS-X': nan,
 'FS-Z': nan,
 'FT%': 6.9,
 'FT-X': 7.5,
 'FT-Z': 9.2,
 'G': 15.0,
 'GB%': 48.4,
 'GB/FB': 1.36,
 'GS': 0.0,
 'H': 10.0,
 'HBP': 0.0,
 'HLD': 2.0,
 'HR': 0.0,
 'HR/9': 0.0,
 'HR/FB': 0.0,
 'Hard%': 21.9,
 'IBB': 0.0,
 'IFFB%': 0.0,
 'IP': 12.1,
 'K%': 30.0,
 'K-BB%': 24.0,
 'K/9': 10.95,
 'K/BB': 5.0,
 'KC%':

In [136]:
p_2014_dict['10855'][cols2014_15[3]]

15.0

In [147]:
for index, row in pitchers_2014_15_avg[1:].iterrows():
    if row['playerid'] in p_2014_dict:
        for n in range(0,len(cols2014_15)):
            if cols2014_15[n] in p_2014_dict[row['playerid']]:
                pitchers_2014_15_avg.loc[index, cols2014_15[n]] = (pitchers_2014_15_avg.loc[index, cols2014_15[n]] - p_2014_dict[row['playerid']][cols2014_15[n]])/(p_2014_dict[row['playerid']][cols2014_15[n]] + 0.00000001)

In [73]:
np.mean([p_2014_dict['1247']['CT%'],p_2014_dict['1247']['CTv']])

nan

In [148]:
pitchers_2014_15_avg.head()

,Unnamed: 0,W,L,G,GS,CG,ShO,SV,HLD,BS,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,playerid,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,LOB%,ERA-,FIP-,xFIP-,FIP,E-F,xFIP,SIERA,GB/FB,LD%,GB%,FB%_x,IFFB%,HR/FB,RS,RS/9,Balls,Strikes,Pitches,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,FA-X,FT-X,FC-X,FS-X,FO-X,SI-X,SL-X,CU-X,KC-X,EP-X,CH-X,SC-X,KN-X,FA-Z,FT-Z,FC-Z,FS-Z,FO-Z,SI-Z,SL-Z,CU-Z,KC-Z,EP-Z,CH-Z,SC-Z,KN-Z,O-Swing%_x,Z-Swing%_x,Swing%_x,O-Contact%_x,Z-Contact%_x,Contact%_x,Zone%_x,Pace,vFA,vFT,vFC,vFS,vFO,vSI,vSL,vCU,vKC,vEP,vCH,vSC,vKN,FA%,FT%,FC%,FS%,FO%,SI%,SL%_x,CU%,KC%,EP%,CH%_x,SC%,KN%_x,UN%,FB%_y,FBv,SL%_y,SLv,CT%,CTv,CB%,CBv,CH%_y,CHv,SF%,SFv,KN%_y,KNv,XX%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,"﻿""Name""",Team,O-Swing%_y,Z-Swing%_y,Swing%_y,O-Contact%_y,Z-Contact%_y,Contact%_y,Zone%_y,F-Strike%,SwStr%,IP,BABIP,ERA
0,0,0.000000,-1.0,-0.866667,0.0,0.0,0.0,2.000000,-8.571429e-01,-1.000000,-0.871324,-0.955224,-1.000000,-1.000000,-1.0,-0.875000,-1.0,-7.500000e-01,-1.000000,0.0,-0.814286,1247,0.168215,-0.216216,0.484018,-1.000000e+00,0.443580,-0.033898,0.577889,-0.646617,-0.625000,0.288660,-1.000000,-0.538462,-0.265060,-0.519355,-3.980000,-0.232026,-0.379310,0.449275,-0.726804,0.353319,-0.067847,2.397959,-1.0,-0.965517,-0.783582,-0.860215,-0.873950,-0.869245,-0.095823,-0.042424,0.196970,0.593939,0.109195,-0.495208,0.125000,-0.042105,NaN,NaN,NaN,NaN,-0.233766,NaN,NaN,NaN,NaN,NaN,NaN,0.355556,0.000000,NaN,NaN,NaN,NaN,-0.416667,NaN,NaN,NaN,NaN,NaN,NaN,0.144654,-0.188776,-0.087719,-0.053004,-0.005924,-0.072096,-0.101961,-0.070312,0.006363,0.024758,NaN,NaN,NaN,NaN,0.016990,NaN,NaN,NaN,NaN,NaN,NaN,-0.205021,3.093023,NaN,NaN,NaN,NaN,-0.138947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.065385,0.014846,-0.165254,0.014545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.750000,-0.463415,NaN,NaN,NaN,NaN,NaN,-23.045456,3.814815,NaN,NaN,NaN,NaN,NaN,Adam Ottavino,Rockies,0.066869,-0.189456,-0.093682,-0.091068,-0.022016,-0.075798,-0.021368,-0.162866,0.123894,-0.844615,-0.544669,-1.000000
1,1,2.000000,0.0,19.000000,0.0,0.0,0.0,0.000000,3.000000e+00,1.000000,102.000000,15.000000,3.000000,2.000000,1.0,11.000000,1.0,1.000000e+00,1.000000,0.0,23.000000,15671,8.390000,4.010000,2.090000,3.600000e-01,22.600000,10.800000,11.800000,0.167000,1.050000,93.800000,19.000000,82.000000,111.000000,3.260000,-2.530000,4.340000,3.860000,1.150000,13.400000,46.300000,40.300000,7.400000,3.7,17.000000,6.200000,159.000000,270.000000,429.000000,34.300000,28.400000,37.300000,10.500000,67.200000,22.400000,-1.200000,-4.900000,0.400000,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,-5.600000,NaN,NaN,11.400000,10.700000,8.500000,NaN,NaN,NaN,0.800000,NaN,NaN,NaN,6.500000,NaN,NaN,28.700000,64.800000,47.100000,66.700000,84.300000,79.000000,50.800000,22.500000,92.400000,93.200000,89.700000,NaN,NaN,NaN,85.400000,NaN,NaN,NaN,86.200000,NaN,NaN,20.200000,43.800000,1.200000,NaN,NaN,NaN,22.800000,NaN,NaN,NaN,12.000000,NaN,NaN,NaN,63.800000,93.100000,23.500000,85.300000,NaN,NaN,NaN,NaN,12.700000,86.600000,NaN,NaN,NaN,NaN,0.9,4.000000,0.800000,NaN,NaN,0.400000,NaN,NaN,1.470000,0.750000,NaN,NaN,0.690000,NaN,NaN,Dalier Hinojosa,- - -,24.800000,72.400000,47.100000,67.900000,84.000000,79.500000,46.800000,54.900000,9.600000,24.200000,0.212000,0.730000
2,2,0.000000,-0.5,-0.793103,0.0,0.0,0.0,-0.666667,-6.000000e-01,-1.000000,-0.795122,-0.787879,-0.941176,-0.937500,-1.0,-0.851852,0.0,0.000000e+00,-0.777778,0.0,-0.806452,3271,-0.063620,-0.283951,0.304348,-1.000000e+00,-0.052980,-0.280303,0.116959,-0.005405,-0.116667,0.208777,-0.708861,-0.333333,-0.101124,-0.293907,-13.222221,-0.051672,-0.032258,0.086614,0.408377,-0.064159,-0.137255,1.551020,-1.0,-0.916667,-0.597222,-0.855072,-0.796935,-0.820069,0.145833,-0.254839,0.090652,-0.049587,0.053140,-0.108108,0.211268,NaN,NaN,NaN,NaN,NaN,-0.375000,NaN,NaN,NaN,0.244186,NaN,NaN,0.021505,NaN,NaN,NaN,NaN,NaN,-5.750000,NaN,NaN,NaN,-0.019608,NaN,NaN,0.358824,0.055380,0.166667,-0.117187,-0.033210,-0.104197,-0.070455,0.004348,-0.015641,NaN,NaN,NaN,NaN,NaN,0.007203,NaN,NaN,N

In [150]:
pitchers_2014[pitchers_2014['playerid'] == 15671]

,Unnamed: 0,W,L,G,GS,CG,ShO,SV,HLD,BS,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,playerid,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,LOB%,ERA-,FIP-,xFIP-,FIP,E-F,xFIP,SIERA,GB/FB,LD%,GB%,FB%_x,IFFB%,HR/FB,RS,RS/9,Balls,Strikes,Pitches,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,FA-X,FT-X,FC-X,FS-X,FO-X,SI-X,SL-X,CU-X,KC-X,EP-X,CH-X,SC-X,KN-X,FA-Z,FT-Z,FC-Z,FS-Z,FO-Z,SI-Z,SL-Z,CU-Z,KC-Z,EP-Z,CH-Z,SC-Z,KN-Z,O-Swing%_x,Z-Swing%_x,Swing%_x,O-Contact%_x,Z-Contact%_x,Contact%_x,Zone%_x,Pace,vFA,vFT,vFC,vFS,vFO,vSI,vSL,vCU,vKC,vEP,vCH,vSC,vKN,FA%,FT%,FC%,FS%,FO%,SI%,SL%_x,CU%,KC%,EP%,CH%_x,SC%,KN%_x,UN%,FB%_y,FBv,SL%_y,SLv,CT%,CTv,CB%,CBv,CH%_y,CHv,SF%,SFv,KN%_y,KNv,XX%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,"﻿""Name""",Team,O-Swing%_y,Z-Swing%_y,Swing%_y,O-Contact%_y,Z-Contact%_y,Contact%_y,Zone%_y,F-Strike%,SwStr%,IP,BABIP,ERA


In [151]:
pitchers_2015 = pd.read_csv('pitchers_2015.csv')
pitchers_2015[pitchers_2015['playerid'] == 15671]

,Unnamed: 0,W,L,G,GS,CG,ShO,SV,HLD,BS,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,playerid,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,LOB%,ERA-,FIP-,xFIP-,FIP,E-F,xFIP,SIERA,GB/FB,LD%,GB%,FB%_x,IFFB%,HR/FB,RS,RS/9,Balls,Strikes,Pitches,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,FA-X,FT-X,FC-X,FS-X,FO-X,SI-X,SL-X,CU-X,KC-X,EP-X,CH-X,SC-X,KN-X,FA-Z,FT-Z,FC-Z,FS-Z,FO-Z,SI-Z,SL-Z,CU-Z,KC-Z,EP-Z,CH-Z,SC-Z,KN-Z,O-Swing%_x,Z-Swing%_x,Swing%_x,O-Contact%_x,Z-Contact%_x,Contact%_x,Zone%_x,Pace,vFA,vFT,vFC,vFS,vFO,vSI,vSL,vCU,vKC,vEP,vCH,vSC,vKN,FA%,FT%,FC%,FS%,FO%,SI%,SL%_x,CU%,KC%,EP%,CH%_x,SC%,KN%_x,UN%,FB%_y,FBv,SL%_y,SLv,CT%,CTv,CB%,CBv,CH%_y,CHv,SF%,SFv,KN%_y,KNv,XX%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,"﻿""Name""",Team,O-Swing%_y,Z-Swing%_y,Swing%_y,O-Contact%_y,Z-Contact%_y,Contact%_y,Zone%_y,F-Strike%,SwStr%,IP,BABIP,ERA
1,1,2,0,19,0,0,0,0,3,1,102,15,3,2,1,11,1,1,1,0,23,15671,8.39,4.01,2.09,0.36,22.6 %,10.8 %,11.8 %,0.167,1.05,93.8 %,19,82,111,3.26,-2.53,4.34,3.86,1.15,13.4 %,46.3 %,40.3 %,7.4 %,3.7 %,17,6.2,159,270,429,34.3 %,28.4 %,37.3 %,10.5 %,67.2 %,22.4 %,-1.2,-4.9,0.4,NaN,NaN,NaN,2.0,NaN,NaN,NaN,-5.6,NaN,NaN,11.4,10.7,8.5,NaN,NaN,NaN,0.8,NaN,NaN,NaN,6.5,NaN,NaN,28.7 %,64.8 %,47.1 %,66.7 %,84.3 %,79.0 %,50.8 %,22.5,92.4,93.2,89.7,NaN,NaN,NaN,85.4,NaN,NaN,NaN,86.2,NaN,NaN,20.2 %,43.8 %,1.2 %,NaN,NaN,NaN,22.8 %,NaN,NaN,NaN,12.0 %,NaN,NaN,NaN,63.8 %,93.1,23.5 %,85.3,NaN,NaN,NaN,NaN,12.7 %,86.6,NaN,NaN,NaN,NaN,0.9 %,4.0,0.8,NaN,NaN,0.4,NaN,NaN,1.47,0.75,NaN,NaN,0.69,NaN,NaN,Dalier Hinojosa,- - -,24.8 %,72.4 %,47.1 %,67.9 %,84.0 %,79.5 %,46.8 %,54.9 %,9.6 %,24.2,0.212,0.73
